In [3]:
import pandas as pd
from matplotlib import pyplot as plt
df = pd.read_csv("churn-data-3333.csv")

In [11]:
num_df = df.select_dtypes(include=['float64', 'int64']).copy()
cat_df = df.select_dtypes(include=['object']).copy()

In [13]:
num_df.head()

,account_length,area_code,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls
0,20.627417,38.743098,8.000000,30.563784,18.976177,11.426839,26.099822,17.899749,6.192680,29.285780,17.078784,4.636264,4.324555,1.464102,1.286335,0.000000
1,18.688161,38.743098,8.198039,23.424398,20.181073,8.482366,25.964263,18.297783,6.153527,29.899843,18.297783,4.767570,5.402702,1.464102,1.847077,0.000000
2,21.409400,38.743098,0.000000,29.202564,19.354157,10.865458,20.018174,18.976177,4.418723,23.502941,18.396078,3.411100,4.985700,2.472136,1.627671,0.000000
3,16.330303,38.398020,0.000000,32.606358,14.852300,12.268847,13.735311,16.761663,2.586938,26.064212,16.867962,3.953150,3.138093,3.291503,0.668333,0.828427
4,15.320508,38.743098,0.000000,23.822471,19.260292,8.647065,22.355697,20.090722,5.102112,25.342275,20.000000,3.800000,4.356099,1.464102,1.304542,1.464102


In [4]:
num_df = df.select_dtypes(include=['float64', 'int64']).copy()
cat_df = df.select_dtypes(include=['object']).copy()

# Categorical boolean mask
categorical_feature_mask = cat_df.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = cat_df.columns[categorical_feature_mask].tolist()

import numpy as np
#conData=np.log(0.00001 + 1)
conData=0
cat_df=cat_df.fillna(conData)
num_df=num_df.fillna(conData)
cat_df=cat_df.astype(str)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_df[categorical_cols] = cat_df[categorical_cols].apply(lambda col: le.fit_transform(col))

#cat_df[categorical_cols].head(10)


churn=cat_df['churn']
cat_df=cat_df.drop(['churn'], axis=1)
cat_df=cat_df.drop(['phone number'], axis=1)



#BOX-COX
lemda=0.5
num_df=(num_df**lemda)
num_df=num_df-1
num_df[num_df < 0]=0
num_df=num_df.div(lemda)
#End BOX-COX
#Z-score
#num_df=(num_df-num_df.min())/(num_df.std(ddof=0)) ##Z-score

#Log
#num_df=round(np.log(num_df.add(1)),2)
#num_df=num_df.replace([np.inf, -np.inf], np.nan)
#End Log

num_df=num_df.fillna(conData)

result_df = pd.concat([num_df, cat_df], axis=1)
np.nan_to_num(result_df)

result_df_op=result_df
#result_df_op=result_df_op.drop(['churn'], axis=1)
#result_df_op=np.nan_to_num(result_df_op)

X=result_df_op
#y=result_df['churn'] 
y=churn

from sklearn.model_selection import train_test_split  
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.0, random_state=42)
X_train=X
X_test=X
y_train=y
y_test=y

In [26]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
#Create a Gaussian Classifier
classifier = GaussianNB()

classifier.fit(X_train,y_train)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

from sklearn.model_selection import cross_val_score
roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
print('roc_auc: ',roc_auc.mean())

from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(y_train, y_pred)
auc = auc(fpr, tpr)
print('auc: ',auc)

2620 230 262 221
pod:  0.4575569358178054
pof:  0.08070175438596491
AUC:  0.6884275907159202
roc_auc:  0.8360651629072681
auc:  0.6884275907159202


In [27]:
# KNN
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(n_neighbors=5)  
classifier.fit(X_train, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

2813 37 387 96
pod:  0.19875776397515527
pof:  0.012982456140350877
AUC:  0.5928876539174022


In [28]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(n_estimators=100,max_depth=2,random_state=0)
classifier.fit(X_train, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

2849 1 473 10
pod:  0.020703933747412008
pof:  0.0003508771929824561
AUC:  0.5101765282772147


In [29]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression()
classifier.fit(X_train, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)   

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

2788 62 397 86
pod:  0.17805383022774326
pof:  0.02175438596491228
AUC:  0.5781497221314155


In [5]:
#SVM
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, y_train)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarni

2848 2 462 21
pod:  0.043478260869565216
pof:  0.0007017543859649122
AUC:  0.5213882532418002


In [30]:
#FNN
def get_FNN_Predict(X2_train, X2_test, y2_train, y2_test):
    from keras.models import Sequential
    from keras.layers import Dense
    #create model
    model = Sequential()
    #get number of columns in training data
    n_cols = X2_train.shape[1]
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    #compile model using mse as a measure of model performance
    #model.compile(optimizer='adam', loss='mean_squared_error')
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  
    from keras.callbacks import EarlyStopping
    #set early stopping monitor so the model stops training when it won't improve anymore
    early_stopping_monitor = EarlyStopping(patience=3)
    #train model
    model.fit(X2_train, y2_train, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])
    y2_pred = model.predict(X2_test)
    print('y2_pred: ',y2_pred)
    y22_pred=y2_pred.round()
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(X_train,y_train):
    X2_train, X2_test, y2_train, y2_test=X_train.iloc[train_index], X_train.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_FNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

print ('tn mean: ',sum(tn_list) / len(tn_list))
print ('fp mean: ',sum(fp_list) / len(fp_list))
print ('fn mean: ',sum(fn_list) / len(fn_list))
print ('tp mean: ',sum(tp_list) / len(tp_list))

Using TensorFlow backend.







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 2399 samples, validate on 600 samples
Epoch 1/30
2399/2399 [==============================] - 1s 241us/step - loss: 0.5054 - acc: 0.8370 - val_loss: 0.4092 - val_acc: 0.8417
Epoch 2/30
2399/2399 [==============================] - 0s 91us/step - loss: 0.4025 - acc: 0.8595 - val_loss: 0.5112 - val_acc: 0.8417
Epoch 3/30
2399/2399 [==============================] - 0s 113us/step - loss: 0.3916 - acc: 0.8566 - val_loss: 0.4231 - val_acc: 0.8417
Epoch 4/30
2399/2399 [==============================] - 0s 95us/step - loss: 0.3849 - acc: 0.8579 - val_loss: 0.4100 - val_acc: 0.8400
y2_pred:  [[0.09379867]
 [0.06402332]
 [0.18342617]
 [0.25084698]
 [0.1547041 ]
 [0.06265712]
 [0.06066841]
 [0.10247794]
 [0.1126135 ]
 [0.05955875]
 [0.14245012]
 [0.11114389]
 [0.10678911]
 [0.11998665]
 [0.12022486]
 [0.20064089]
 [0.14145634]
 [0.09103647]
 [0.05215603]
 [0.17149708]
 [0.07669172]
 [0.059

Train on 2399 samples, validate on 600 samples
Epoch 1/30
2399/2399 [==============================] - 1s 261us/step - loss: 0.5921 - acc: 0.8387 - val_loss: 0.4340 - val_acc: 0.8417
Epoch 2/30
2399/2399 [==============================] - 0s 110us/step - loss: 0.4099 - acc: 0.8587 - val_loss: 0.4289 - val_acc: 0.8417
Epoch 3/30
2399/2399 [==============================] - 0s 108us/step - loss: 0.4011 - acc: 0.8587 - val_loss: 0.4558 - val_acc: 0.8417
Epoch 4/30
2399/2399 [==============================] - 0s 106us/step - loss: 0.4018 - acc: 0.8570 - val_loss: 0.4257 - val_acc: 0.8450
Epoch 5/30
2399/2399 [==============================] - 0s 113us/step - loss: 0.3982 - acc: 0.8574 - val_loss: 0.4122 - val_acc: 0.8433
Epoch 6/30
2399/2399 [==============================] - 0s 123us/step - loss: 0.3977 - acc: 0.8558 - val_loss: 0.4404 - val_acc: 0.8417
Epoch 7/30
2399/2399 [==============================] - 0s 131us/step - loss: 0.3947 - acc: 0.8595 - val_loss: 0.4411 - val_acc: 0.8500
E

Train on 2399 samples, validate on 600 samples
Epoch 1/30
2399/2399 [==============================] - 1s 420us/step - loss: 0.4313 - acc: 0.8520 - val_loss: 0.4198 - val_acc: 0.8450
Epoch 2/30
2399/2399 [==============================] - 0s 109us/step - loss: 0.3952 - acc: 0.8629 - val_loss: 0.4059 - val_acc: 0.8517
Epoch 3/30
2399/2399 [==============================] - 0s 116us/step - loss: 0.3841 - acc: 0.8662 - val_loss: 0.4258 - val_acc: 0.8417
Epoch 4/30
2399/2399 [==============================] - 0s 113us/step - loss: 0.3830 - acc: 0.8695 - val_loss: 0.4188 - val_acc: 0.8533
Epoch 5/30
2399/2399 [==============================] - 0s 130us/step - loss: 0.3788 - acc: 0.8704 - val_loss: 0.3872 - val_acc: 0.8667
Epoch 6/30
2399/2399 [==============================] - 0s 105us/step - loss: 0.3640 - acc: 0.8770 - val_loss: 0.3809 - val_acc: 0.8617
Epoch 7/30
2399/2399 [==============================] - 0s 114us/step - loss: 0.3537 - acc: 0.8825 - val_loss: 0.4009 - val_acc: 0.8450
E

Train on 2400 samples, validate on 600 samples
Epoch 1/30
2400/2400 [==============================] - 1s 411us/step - loss: 0.4866 - acc: 0.8458 - val_loss: 0.4664 - val_acc: 0.8417
Epoch 2/30
2400/2400 [==============================] - 0s 111us/step - loss: 0.4348 - acc: 0.8583 - val_loss: 0.4458 - val_acc: 0.8417
Epoch 3/30
2400/2400 [==============================] - 0s 109us/step - loss: 0.4184 - acc: 0.8583 - val_loss: 0.4430 - val_acc: 0.8417
Epoch 4/30
2400/2400 [==============================] - 0s 114us/step - loss: 0.4144 - acc: 0.8583 - val_loss: 0.4361 - val_acc: 0.8417
Epoch 5/30
2400/2400 [==============================] - 0s 110us/step - loss: 0.4108 - acc: 0.8583 - val_loss: 0.4348 - val_acc: 0.8417
Epoch 6/30
2400/2400 [==============================] - 0s 109us/step - loss: 0.4102 - acc: 0.8583 - val_loss: 0.4308 - val_acc: 0.8417
Epoch 7/30
2400/2400 [==============================] - 0s 101us/step - loss: 0.4030 - acc: 0.8583 - val_loss: 0.4259 - val_acc: 0.8417
E

...

Train on 2400 samples, validate on 600 samples
Epoch 1/30
2400/2400 [==============================] - 1s 456us/step - loss: 0.4183 - acc: 0.8583 - val_loss: 0.4352 - val_acc: 0.8417
Epoch 2/30
2400/2400 [==============================] - 0s 120us/step - loss: 0.4073 - acc: 0.8583 - val_loss: 0.4347 - val_acc: 0.8417
Epoch 3/30
2400/2400 [==============================] - 0s 133us/step - loss: 0.4002 - acc: 0.8583 - val_loss: 0.4232 - val_acc: 0.8417
Epoch 4/30
2400/2400 [==============================] - 0s 122us/step - loss: 0.3964 - acc: 0.8583 - val_loss: 0.4195 - val_acc: 0.8417
Epoch 5/30
2400/2400 [==============================] - 0s 123us/step - loss: 0.3913 - acc: 0.8583 - val_loss: 0.4530 - val_acc: 0.8417
Epoch 6/30
2400/2400 [==============================] - 0s 123us/step - loss: 0.3974 - acc: 0.8583 - val_loss: 0.4170 - val_acc: 0.8417
Epoch 7/30
2400/2400 [==============================] - 0s 124us/step - loss: 0.3855 - acc: 0.8579 - val_loss: 0.4147 - val_acc: 0.8467
E

Train on 2400 samples, validate on 600 samples
Epoch 1/30
2400/2400 [==============================] - 1s 609us/step - loss: 0.4476 - acc: 0.8500 - val_loss: 0.4298 - val_acc: 0.8417
Epoch 2/30
2400/2400 [==============================] - 0s 144us/step - loss: 0.4006 - acc: 0.8571 - val_loss: 0.4236 - val_acc: 0.8417
Epoch 3/30
2400/2400 [==============================] - 0s 138us/step - loss: 0.3898 - acc: 0.8579 - val_loss: 0.4223 - val_acc: 0.8467
Epoch 4/30
2400/2400 [==============================] - 0s 115us/step - loss: 0.3977 - acc: 0.8575 - val_loss: 0.4065 - val_acc: 0.8417
Epoch 5/30
2400/2400 [==============================] - 0s 110us/step - loss: 0.3876 - acc: 0.8579 - val_loss: 0.3918 - val_acc: 0.8433
Epoch 6/30
2400/2400 [==============================] - 0s 120us/step - loss: 0.3758 - acc: 0.8596 - val_loss: 0.3795 - val_acc: 0.8500
Epoch 7/30
2400/2400 [==============================] - 0s 123us/step - loss: 0.3675 - acc: 0.8612 - val_loss: 0.3727 - val_acc: 0.8600
E

Train on 2400 samples, validate on 600 samples
Epoch 1/30
2400/2400 [==============================] - 1s 493us/step - loss: 0.4592 - acc: 0.8583 - val_loss: 0.4373 - val_acc: 0.8417
Epoch 2/30
2400/2400 [==============================] - 0s 109us/step - loss: 0.4175 - acc: 0.8583 - val_loss: 0.4697 - val_acc: 0.8417
Epoch 3/30
2400/2400 [==============================] - 0s 110us/step - loss: 0.4179 - acc: 0.8583 - val_loss: 0.4266 - val_acc: 0.8417
Epoch 4/30
2400/2400 [==============================] - 0s 99us/step - loss: 0.4023 - acc: 0.8583 - val_loss: 0.4209 - val_acc: 0.8417
Epoch 5/30
2400/2400 [==============================] - 0s 108us/step - loss: 0.3979 - acc: 0.8583 - val_loss: 0.4059 - val_acc: 0.8417
Epoch 6/30
2400/2400 [==============================] - 0s 97us/step - loss: 0.4033 - acc: 0.8583 - val_loss: 0.4013 - val_acc: 0.8417
Epoch 7/30
2400/2400 [==============================] - 0s 116us/step - loss: 0.3970 - acc: 0.8588 - val_loss: 0.3971 - val_acc: 0.8417
Epo

Train on 2400 samples, validate on 600 samples
Epoch 1/30
2400/2400 [==============================] - 1s 483us/step - loss: 0.4846 - acc: 0.8512 - val_loss: 0.5269 - val_acc: 0.8233
Epoch 2/30
2400/2400 [==============================] - 0s 107us/step - loss: 0.4083 - acc: 0.8629 - val_loss: 0.4527 - val_acc: 0.8233
Epoch 3/30
2400/2400 [==============================] - 0s 106us/step - loss: 0.3932 - acc: 0.8629 - val_loss: 0.4509 - val_acc: 0.8233
Epoch 4/30
2400/2400 [==============================] - 0s 111us/step - loss: 0.3983 - acc: 0.8625 - val_loss: 0.4465 - val_acc: 0.8233
Epoch 5/30
2400/2400 [==============================] - 0s 111us/step - loss: 0.3792 - acc: 0.8625 - val_loss: 0.4450 - val_acc: 0.8233
Epoch 6/30
2400/2400 [==============================] - 0s 109us/step - loss: 0.3906 - acc: 0.8612 - val_loss: 0.4229 - val_acc: 0.8317
Epoch 7/30
2400/2400 [==============================] - 0s 107us/step - loss: 0.3727 - acc: 0.8721 - val_loss: 0.4308 - val_acc: 0.8300
E

Train on 2400 samples, validate on 600 samples
Epoch 1/30
2400/2400 [==============================] - 1s 485us/step - loss: 0.4796 - acc: 0.8517 - val_loss: 0.5019 - val_acc: 0.8233
Epoch 2/30
2400/2400 [==============================] - 0s 111us/step - loss: 0.4286 - acc: 0.8629 - val_loss: 0.5053 - val_acc: 0.8233
Epoch 3/30
2400/2400 [==============================] - 0s 106us/step - loss: 0.4009 - acc: 0.8629 - val_loss: 0.4707 - val_acc: 0.8233
Epoch 4/30
2400/2400 [==============================] - 0s 106us/step - loss: 0.3993 - acc: 0.8629 - val_loss: 0.4652 - val_acc: 0.8233
Epoch 5/30
2400/2400 [==============================] - 0s 111us/step - loss: 0.4003 - acc: 0.8629 - val_loss: 0.4854 - val_acc: 0.8233
Epoch 6/30
2400/2400 [==============================] - 0s 107us/step - loss: 0.4004 - acc: 0.8629 - val_loss: 0.4647 - val_acc: 0.8233
Epoch 7/30
2400/2400 [==============================] - 0s 109us/step - loss: 0.3950 - acc: 0.8629 - val_loss: 0.4529 - val_acc: 0.8233
E

In [31]:
#RNN
def get_RNN_Predict(X2_train, X2_test, y2_train, y2_test):
    import pandas as pd
    from keras.models import Sequential
    from keras.layers import Dense,Dropout, LSTM, GRU
    from keras.layers import Embedding
    max_features = 10000 # number of words to consider as features
    import numpy as np
    #create model 
    model = Sequential()
    model.add(Embedding(max_features, 32))
    model.add(LSTM(32))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
    history = model.fit(X2_train, y2_train, epochs=5, batch_size=128, validation_split=0.2)
    y2_pred = model.predict(X2_test)
    y22_pred=y2_pred.round()
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(X_train,y_train):
    X2_train, X2_test, y2_train, y2_test=X_train.iloc[train_index], X_train.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_RNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

print ('tn mean: ',sum(tn_list) / len(tn_list))
print ('fp mean: ',sum(fp_list) / len(fp_list))
print ('fn mean: ',sum(fn_list) / len(fn_list))
print ('tp mean: ',sum(tp_list) / len(tp_list))

Train on 2399 samples, validate on 600 samples
Epoch 1/5
2399/2399 [==============================] - 2s 758us/step - loss: 0.5134 - acc: 0.8587 - val_loss: 0.4360 - val_acc: 0.8417
Epoch 2/5
2399/2399 [==============================] - 0s 181us/step - loss: 0.4048 - acc: 0.8587 - val_loss: 0.4262 - val_acc: 0.8417
Epoch 3/5
2399/2399 [==============================] - 0s 178us/step - loss: 0.3973 - acc: 0.8587 - val_loss: 0.4218 - val_acc: 0.8417
Epoch 4/5
2399/2399 [==============================] - 0s 172us/step - loss: 0.3901 - acc: 0.8587 - val_loss: 0.4118 - val_acc: 0.8417
Epoch 5/5
2399/2399 [==============================] - 0s 172us/step - loss: 0.3868 - acc: 0.8587 - val_loss: 0.4063 - val_acc: 0.8417
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0

Train on 2400 samples, validate on 600 samples
Epoch 1/5
2400/2400 [==============================] - 2s 931us/step - loss: 0.5222 - acc: 0.8538 - val_loss: 0.4501 - val_acc: 0.8417
Epoch 2/5
2400/2400 [==============================] - 0s 178us/step - loss: 0.4053 - acc: 0.8583 - val_loss: 0.4267 - val_acc: 0.8417
Epoch 3/5
2400/2400 [==============================] - 0s 183us/step - loss: 0.3981 - acc: 0.8583 - val_loss: 0.4445 - val_acc: 0.8417
Epoch 4/5
2400/2400 [==============================] - 0s 189us/step - loss: 0.3924 - acc: 0.8583 - val_loss: 0.4271 - val_acc: 0.8417
Epoch 5/5
2400/2400 [==============================] - 0s 205us/step - loss: 0.3805 - acc: 0.8583 - val_loss: 0.4035 - val_acc: 0.8417
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0

Train on 2400 samples, validate on 600 samples
Epoch 1/5
2400/2400 [==============================] - 3s 1ms/step - loss: 0.4988 - acc: 0.8583 - val_loss: 0.4338 - val_acc: 0.8417
Epoch 2/5
2400/2400 [==============================] - 0s 195us/step - loss: 0.4017 - acc: 0.8583 - val_loss: 0.4260 - val_acc: 0.8417
Epoch 3/5
2400/2400 [==============================] - 0s 196us/step - loss: 0.3936 - acc: 0.8583 - val_loss: 0.4154 - val_acc: 0.8417
Epoch 4/5
2400/2400 [==============================] - 0s 192us/step - loss: 0.3886 - acc: 0.8583 - val_loss: 0.4139 - val_acc: 0.8417
Epoch 5/5
2400/2400 [==============================] - 0s 192us/step - loss: 0.3803 - acc: 0.8583 - val_loss: 0.4098 - val_acc: 0.8417
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]

Train on 2400 samples, validate on 600 samples
Epoch 1/5
2400/2400 [==============================] - 4s 2ms/step - loss: 0.4995 - acc: 0.8625 - val_loss: 0.4751 - val_acc: 0.8233
Epoch 2/5
2400/2400 [==============================] - 1s 252us/step - loss: 0.3936 - acc: 0.8629 - val_loss: 0.4564 - val_acc: 0.8233
Epoch 3/5
2400/2400 [==============================] - 1s 285us/step - loss: 0.3916 - acc: 0.8629 - val_loss: 0.4499 - val_acc: 0.8233
Epoch 4/5
2400/2400 [==============================] - 1s 257us/step - loss: 0.3823 - acc: 0.8629 - val_loss: 0.4409 - val_acc: 0.8233
Epoch 5/5
2400/2400 [==============================] - 1s 294us/step - loss: 0.3742 - acc: 0.8629 - val_loss: 0.4414 - val_acc: 0.8233
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]

In [32]:
#CNN - 1D CNN
def get_CNN_Predict(X2_train, X2_test, y2_train, y2_test):
    from keras.models import Sequential
    from keras import layers
    from keras.optimizers import RMSprop
    max_features = 10000 # number of words to consider as features
    
    model = Sequential()
    #model.add(layers.Embedding(max_features, 128, input_length=96))
    model.add(layers.Embedding(max_features, 128, input_length=19))
    model.add(layers.Conv1D(32, 2, activation='relu'))
    model.add(layers.MaxPooling1D(5))
    model.add(layers.Conv1D(32, 2, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(1))
    model.summary()
    model.compile(optimizer=RMSprop(lr=1e-4), loss='binary_crossentropy', metrics=['acc'])
    history = model.fit(X2_train, y2_train, epochs=5, batch_size=128, validation_split=0.2)
    y2_pred = model.predict(X2_test)
    y22_pred=y2_pred.round()
    y22_pred[y22_pred ==2.0]=1
   # y22_pred=y22_pred[y22_pred > 1.0]=1
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(X_train,y_train):
    X2_train, X2_test, y2_train, y2_test=X_train.iloc[train_index], X_train.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_CNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

print ('tn mean: ',sum(tn_list) / len(tn_list))
print ('fp mean: ',sum(fp_list) / len(fp_list))
print ('fn mean: ',sum(fn_list) / len(fn_list))
print ('tp mean: ',sum(tp_list) / len(tp_list))



Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 19, 128)           1280000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 18, 32)            8224      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 3, 32)             0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 2, 32)             2080      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_61 (Dense)             (None, 1)                 33        
Total params: 1,290,337
Trainable params: 1,290,337
Non-trainable params: 0
__________________________________________

Train on 2399 samples, validate on 600 samples
Epoch 1/5
2399/2399 [==============================] - 3s 1ms/step - loss: 1.7803 - acc: 0.8587 - val_loss: 0.6518 - val_acc: 0.8417
Epoch 2/5
2399/2399 [==============================] - 1s 266us/step - loss: 0.5022 - acc: 0.8587 - val_loss: 0.5286 - val_acc: 0.8417
Epoch 3/5
2399/2399 [==============================] - 1s 299us/step - loss: 0.4444 - acc: 0.8587 - val_loss: 0.4726 - val_acc: 0.8417
Epoch 4/5
2399/2399 [==============================] - 1s 300us/step - loss: 0.4201 - acc: 0.8587 - val_loss: 0.4490 - val_acc: 0.8417
Epoch 5/5
2399/2399 [==============================] - 1s 307us/step - loss: 0.4064 - acc: 0.8587 - val_loss: 0.4369 - val_acc: 0.8417
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]

2400/2400 [==============================] - 1s 328us/step - loss: 2.2781 - acc: 0.8583 - val_loss: 2.5373 - val_acc: 0.8417
Epoch 4/5
2400/2400 [==============================] - 1s 295us/step - loss: 2.2780 - acc: 0.8583 - val_loss: 2.5343 - val_acc: 0.8417
Epoch 5/5
2400/2400 [==============================] - 1s 307us/step - loss: 2.2470 - acc: 0.8583 - val_loss: 2.3322 - val_acc: 0.8417
y22_pred:  [[-0.]
 [-0.]
 [-0.]
 [-0.]
 [ 0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [ 0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [ 0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [ 0.]
 [-0.]
 [ 0.]
 [ 0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [ 0.]
 [ 0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [ 0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [ 0.]
 [ 0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]